Salency maps para NLP utilizando AllenNLP
=========================================

Introducción
------------

AllenNLP es un framework general de aprendizaje profundo para NLP, establecido por el mundialmente famoso Allen Institute for AI Lab. Contiene modelos de referencia de última generación que se ejecutan sobre el `PyTorch`. AllenNLP es una librería que ademas busca implementar abstracciones que permitan el rápido desarrollo de modelos y reutilización de componentes al despegarse de detalles de implementación de cada modelo.

En este ejemplo, veremos como utilizar esta librería para generar salency maps utilizando los gradientes de las prediciones. Esto nos permita interpretar las predicciones de nuestros modelos basados en `transformers`.

### Para ejecutar este notebook

Para ejecutar este notebook, instale las siguientes librerias:

In [1]:
!wget https://raw.githubusercontent.com/santiagxf/M72109/master/NLP/Datasets/mascorpus/tweets_marketing.csv \
    --quiet --no-clobber --directory-prefix ./Datasets/mascorpus/

!wget https://raw.githubusercontent.com/santiagxf/M72109/master/NLP/Utils/TextDataset.py \
    --quiet --no-clobber --directory-prefix ./Utils/
    
!wget https://raw.githubusercontent.com/santiagxf/M72109/master/docs/nlp/neural/allennlp_interpret.txt \
    --quiet --no-clobber
!pip install -r allennlp_interpret.txt --quiet

Si ejecuta en Google Colab, adicionalmente deberá:

In [ ]:
!pip install -U google-cloud-storage==1.40.0

Descargaremos un modelo previamente entrenando el el problema de clasificación de Tweets:

In [347]:
!wget https://santiagxf.blob.core.windows.net/public/models/tweet_classification_bert.zip --no-clobber --quiet
!unzip tweet_classification_bert.zip -qq

Cargamos el set de datos

In [2]:
import pandas as pd

tweets = pd.read_csv('Datasets/mascorpus/tweets_marketing.csv')

Cargando un modelo entreando con Transformers en AllenNLP
---------------------------------------------------------

`allennlp` es un framework compatible con la libraría `transformers` lo cual resulta atractivo a la hora de utilizar modelos que son entrenados en una para luego llevarlo a la otra. Veamos entonces como podemos hacer para cargar el modelo que tenemos previamente entrenado para la clasificación de tweets utilizando una arquitectura `BERT` dentro de este framework. En particular, nuestro modelo se persistió en el directorio "tweet_classification".

In [290]:
model_name = "tweet_classification"

### Creando un objeto Model

Importamos algunos elementos que necesitaremos

In [341]:
from typing import Dict, Iterable, List

from allennlp.common import Params
from allennlp.data import DatasetReader, Instance, Batch
from allennlp.data.fields import Field, LabelField, TextField
from allennlp.data.token_indexers import TokenIndexer
from allennlp.data.tokenizers import Tokenizer
from allennlp.data.vocabulary import PreTrainedTokenizer, Vocabulary
from allennlp.models import BasicClassifier, Model
from allennlp.modules.token_embedders import PretrainedTransformerEmbedder
from allennlp.modules.text_field_embedders import BasicTextFieldEmbedder
from allennlp.data.tokenizers.pretrained_transformer_tokenizer import PretrainedTransformerTokenizer
from allennlp.data.token_indexers.pretrained_transformer_indexer import PretrainedTransformerIndexer
from allennlp.modules.seq2vec_encoders.bert_pooler import BertPooler

Cargaremos todos los elementos que son necesarios para utilizar esta libreria. Todos ellos son generados a partir del modelo que persistimos en `transformers`. La utilidad de cada uno de estos módulos esta fuera del alcance de este curso pero recomendamos revisar la documentación de AllenNLP para más información sobre cual es su rol.

In [147]:
transformer_vocab = Vocabulary.from_pretrained_transformer(model_name)
transformer_tokenizer = PretrainedTransformerTokenizer(model_name)
transformer_encoder = BertPooler(model_name)

token_indexer = PretrainedTransformerIndexer(model_name)

In [144]:
params = Params(
    {
     "token_embedders": {
        "tokens": {
          "type": "pretrained_transformer",
          "model_name": model_name,
        }
      }
    }
)

token_embedder = BasicTextFieldEmbedder.from_params(vocab=transformer_vocab, params=params)

Creadmos el modelo a partir de todos los componentes que cargamos

In [148]:
model = BasicClassifier(vocab=transformer_vocab, text_field_embedder=token_embedder, seq2vec_encoder=transformer_encoder, dropout=0.1, num_labels=7)

### Creamos un DatasetReader

AllenNLP utiliza un objeto llamado `DatasetReader` que le permite crear `Instance`'s de datos que son suministradas al modelo. Esta abstracción permite realizar cualquier preprocesamiento que es necesario antes de enviar los datos al modelo. Debemos generar nuestro propia implementación para el caso de clasificación utilizando un modelo basado en *transformers*. La siguiente clase realiza esto: 

In [151]:
from allennlp.data import DatasetReader

In [153]:
class ClassificationTransformerReader(DatasetReader):
    def __init__(
        self,
        tokenizer: Tokenizer,
        token_indexer: TokenIndexer,
        max_tokens: int,
        **kwargs
    ):
        super().__init__(**kwargs)
        self.tokenizer = tokenizer
        self.token_indexers: Dict[str, TokenIndexer] = { "tokens": token_indexer }
        self.max_tokens = max_tokens
        self.vocab = vocab

    def text_to_instance(self, text: str, label: str = None) -> Instance:
        tokens = self.tokenizer.tokenize(text)
        if self.max_tokens:
            tokens = tokens[: self.max_tokens]
        
        inputs = TextField(tokens, self.token_indexers)
        fields: Dict[str, Field] = { "tokens": inputs }
            
        if label:
            fields["label"] = LabelField(label)
            
        return Instance(fields)

Instanciamos el `DatasetReader`

In [155]:
dataset_reader = ClassificationTransformerReader(tokenizer=transformer_tokenizer, token_indexer=token_indexer, max_tokens=400)

### Provemos que nuestro modelo funciona

Unamos todas las piezas que generamos hasta el momento probandolo sobre una instancia:

In [307]:
sample_text = "No supimos como salir del supermercado luego de tantas vueltas"
instance = dataset_reader.text_to_instance(sample_text)

In [308]:
from allennlp.nn import util

dataset = Batch([instance])
dataset.index_instances(transformer_vocab)
model_input = util.move_to_device(dataset.as_tensor_dict(), model._get_prediction_device())

In [319]:
outputs = model.make_output_human_readable(model(**model_input))
print(outputs['probs'].argmax())

tensor(4)

Recordemos que en el conjunto de datos de entrenamiento, las etiquetas se distribuyen como sigue:

```
{
    'ALIMENTACION': 0,
    'AUTOMOCION': 1,
    'BANCA': 2,
    'BEBIDAS': 3,
    'DEPORTES': 4,
    'RETAIL': 5,
    'TELCO': 6
}
```

Interpretando nuestras predicciones
-----------------------------------

Una vez que tenemos nuestro modelo correctamente cargado, veamos como podemos interpretar una predicción computando el salency map a partir de los gradientes.

In [283]:
from allennlp.interpret.saliency_interpreters import SimpleGradient
from allennlp.predictors import Predictor, TextClassifierPredictor

predictor = TextClassifierPredictor(model, dataset_reader)
interpreter = SimpleGradient(predictor)

Busquemos un tweet para interpretar:

In [306]:
sample_text_idx = 2071
sample_text = tweets['TEXTO'][sample_text_idx]
sample_label = tweets['SECTOR'][sample_text_idx]

print("Texto:", sample_text, "\Sector:", sample_label)

Samples: BBVA remolca el crecimiento de la banca española pese a los obstáculos https://t.co/wF9tOqxB5D 
Label: BANCA


Calculemos los gradientes para cada token:

In [285]:
inputs = {"sentence": sample_text }

In [286]:
interpretation = interpreter.saliency_interpret_from_json(inputs)
grads = interpretation['instance_1']['grad_input_1']

Podemos graficar los resultados utilizando un mapa de calor marcando con colores más intensos aquellos tokens que tienen mayor impacto en las predicciones:

In [289]:
import math
from IPython.display import HTML

html = ""
for idx, token in enumerate(tokenizer.tokenize(inputs['sentence'])):
    html += "<span style='background-color:rgba(255,0,0,{})'>{} </span>".format(grads[idx],token)
    
HTML(html)